In [ ]:
pip install google

In [ ]:
pip install beautifulsoup4

In [ ]:
import nltk
nltk.download('words')

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [ ]:
import requests
from bs4 import BeautifulSoup
from googlesearch import search
import webbrowser
from nltk.corpus import words
import math
import sys
import random

In [ ]:
def web_scraping(qs):
    global flag2
    global loading

    URL = 'https://www.google.com/search?q=' + qs
    page = requests.get(URL)

    soup = BeautifulSoup(page.content, 'html.parser')
    links = soup.findAll("a")
    all_links = []
    for link in links:
       link_href = link.get('href')
       if "url?q=" in link_href and not "webcache" in link_href:
           all_links.append((link.get('href').split("?q=")[1].split("&sa=U")[0]))
           
    
    flag= False
    for link in all_links:
       if 'https://en.wikipedia.org/wiki/' in link:
           wiki = link
           flag = True
           break

    div0 = soup.find_all('div',class_="kvKEAb")
    div1 = soup.find_all("div", class_="Ap5OSd")
    div2 = soup.find_all("div", class_="nGphre")
    div3  = soup.find_all("div", class_="BNeawe iBp4i AP7Wnd")

    if len(div0)!=0:
        answer = div0[0].text
    elif len(div1) != 0:
       answer = div1[0].text+"\n"+div1[0].find_next_sibling("div").text
    elif len(div2) != 0:
       answer = div2[0].find_next("span").text+"\n"+div2[0].find_next("div",class_="kCrYT").text
    elif len(div3)!=0:
        answer = div3[1].text
    elif flag==True:
       page2 = requests.get(wiki)
       soup = BeautifulSoup(page2.text, 'html.parser')
       title = soup.select("#firstHeading")[0].text
       
       paragraphs = soup.select("p")
       for para in paragraphs:
           if bool(para.text.strip()):
               answer = title + "\n" + para.text
               break
    else:
        query = qs
        results=[]
        for j in search(query, tld="co.in", num=1, stop=1):
          results.append(j)
        answer = results[0]
        print("Sorry. I could not find the desired results but you can go through this website for more reference")

    return answer

In [ ]:

greet_happy_responses=["You are happy today? Tell me,what's going on",
                       "Hmm,I like that curve on your face?Do you wanna discuss with me what's going on",
                       "You look very happy?How was your day",
                       "You look so exicted and happy.Is anything intresting happened today",
                       "woooo you have a great smile that everyone in the room can fall in love with it",
                       "You have the most beautiful smile in the world",
                       "Seems that you are in a good mood today",
                       ]
greet_neutral_responses=["It's good to see you. How was your day?",
                         "It's nice meeting you. How was it going?"]
greet_angry_responses=["I get that you are angry at something,I hear that anger.Do you wanna discuss about it",
                       "I can see you're feeling angry, do you want to talk about it?",
                       "I can sense that you are angry with something.Do you wanna discuss what's going on?"
                      ]
more_angry_responses=["Okay!just let it out and I'll promise that I will be a good listener",
                     "It's okay!Calm down, Just let that anger out",
                     "Come on, speak to me and let your anger out"]
greet_sad_responses={"You lack the prettiest thing in your face":"Your smile is the prettiest thing in your face and you are missing that",
                     "Seems that you are not in a good mood today":"I think things are not going in your way",
                    "I think you can light up this room with your smile?Do you wanna try it":"Okay we will try some other time",
                    "Smiling makes you look younger than what you are.Do you know that?":"It's okay, no problem.You can try now by smiling"}
more_sad_responses=["Just discuss what is going on with you and I promise I will be a good listener",
                   "Describe me your day, how you have spent and the situations where your mood had changed",
                   "Talk to me about your day,what are the places you have visited and the situations where you lost your smile"]
dont_know_words=["i don't know","don't know","no idea","idk"]
dont_know_sad=["Can't pinpoint why you're sad?Experts suggest writting can make you think clear.Try writing"]
but_words=["It's totally understandable.Do you want to try some techniques which will make you feel better?",
          "I heard that and it's okay to feel this way.Will you try some techniques by which you can feel better?",
          "That's completely fine and it's totally understanbale. Will you try some techniques which will make you feel better"]
techniques={"1.Find what does make you happy.(And laugh).":"you can engage in an activity or hobby you truly enjoy,Even better? Doing something that'll make you laugh like listening to a comedy podcast, or even watching a cat video on YouTube.Laughter releases endorphins similar to exercise, reduces the stress hormone cortisol, and increases dopamine",
           "2.Reach out to the people":"Do things outside the home that include other people,For example, pick something that generally interests you, like a book club. “You’ll be surprised how quickly a community forms.",
           "3.Spend time in nature.":"Experience the outdoors with all five of your senses,Pay attention to what you see, feel, hear, smell and possibly taste in nature, and it may help you out of your slump.According to a study,spending just 17 minutes per day)exploring your local park or walking around your neighborhood can greatly enhance your overall sense of well-being.",
           "4.Get to know yourself":"Sometimes the best thing to do when we’re sad is to go inwards and stop avoiding the sadness.Sit yourself down with some coffee or tea, a notebook, and have a conversation with yourself.Concentrate on self growth",
           "5.Do something you've always wanted to do":"Be a little adventurous.Is there something you’ve wanted to do lately or some place you’ve always wanted to go but never made the time to go there? Well, now’s your chance to do it! Experiencing something new is a great way to put the sadness behind you and put life into a new perspective. Go on an adventure!",
           "6.Write about what you love about yourself":"When you’re sad, it can be really helpful to write a list about everything you love about yourself.Remind yourself of these things everyday while building onto that list. Recognize the good – and remember what an amazing person you are, no matter what may be happening to you right now. You are amazing."}
depressed_activities={"1.Call a friend/family member":"Depression can tempt you to isolate yourself and withdraw from your friends and family, but face-to-face time can help wash away those tendencies.Try to remind yourself these people care about you. Resist the temptation to feel like you’re a burden. You need the interaction — and they likely do, too.Just call your loved ones and inite them for lunch/coffee.Sounds great right",
                     "2.Go for a walk":"Exercising/Walking is as effective as antidepressants and they are considered to be an natural treatment to fight depression So why wait, move your muscle and change your thoughts",
                     "3.Take a long shower":"You know what,Due to the skin’s high density of nerve receptors cold showers are considered to be somewhat a way to send electrical impulses from nerve ending to the brain leading to relief from symptoms of depression. Moreso, cold water natural helps the body to produce endorphins into the blood stream and brain that lifts our mood.So come on just go for it",
                     "4.Do yoga /exercise":"Good one! Move a muscle and change a thought. Changing thoughts and behaviour is key to feeling better.Initiate positive healthy behaviour that help you feel better",
                     "5.Hug a pet if you have one":"That's a great choice. A pet can remind you that you're not alone.Pets offer unconditional love, which can be extraordinarily soothing when feeling isolated.",
                     "6.Do the opposite of what the ‘depression voice’ suggests":"Woo you are in the right path dear, The negative, irrational voice in your head may talk you out of self-help. say to yourself, “You might be right, but it’ll be better than just sitting here another night.” You may soon see the negative isn’t always realistic.",
                      "7.Try something new entirely":"It is one of the best choice you can ever make. When you do the same thing day after day, you use the same parts of your brain. You can challenge your neurons and alter your brain chemistry by doing something entirely different. consider trying a new sport, taking a creative class, or learning a new cooking technique and I'm sure it will definately work"}
anger_activities={"1.Take a deep breathe":"Excellent! Breathing techniques will always help a person to keep the mind and soul in a very relaxed position. If you take a moment to concentrate on your breathing rather than your anger, you will definately have something else to focus on",
                 "2.LOL(Laugh Out Loud)":"Great! Try to laugh louder as much as you can. Experts suggest that laughter enhances your intake of oxygen-rich air, stimulates your heart, lungs and muscles, and increases the endorphins that are released by your brain which can help reduce some of the physical symptoms of anger and stress.",
                 "3.Muscle Relaxation":"Move a muscle and change your thought. Yoga and exercising can be a better medicine for all of your problems. Just start exercising from today",
                 "4.Processing anger calmly":"Let me explain you clearly, Imagine how your day and life will be after 1 day,1 week, 1 month and 1 year. You will definately forget about this issue. By then, you don't even remember this situation anymore. So stay calm and lead your whole day as if this incident never occured",
                 "5.Counting one to ten":"This is one of the famous and old technique for anger management but definately work all the time. Just count one to ten digits calmly and you can extend counting digits upto 20 if your anger is not reduced. Try it once and you will be amazed by the results"}
quiz=["How many times do you feel hungry in a day? (Type number (If you don’t feel hungry then type 0 and if you feel more than thrice just type 3))",
     "Are you sleeping peacfully at night? (yes-3, no-1, sometimes-2)",
     "Do you find interest on the things which made you happier before? (yes-3, no-1, sometimes-2)",
     "How often do you make time to spend with your friends/family? (Regularly-3/none-1/sometimes-2)",
     "Did you find motivation to do things which you are supposed to do?(yes-3, no-1, sometimes-2)"]
more_happy_responses=["Come on discuss more about your day and the things which made you happy",
                "Do you wanna discuss more about your day.Go on, you have my ear",
                "I love to hear more about your day. Come on, talk to me"]
positive=["yes","yeah","ok","okay","fine","no issues","cool","done","correct","no worries","good","nice","it's good","it's nice","exactly"]
negative=["no","nothing","nope","that's it","not","never","none of your business","why do you care","why i have to tell you","i dont want to discuss with you"]
wh_words=['what', 'when', 'where', 'whom', 'which', 'whose', 'why']
about_chatbot_name=["what is your name","who are you","May I know your name","name","May I know with whom I am speaking to"]
about_chatbot_fine=["how are you","how are you doing"]
about_chatbot_boss=["who is your boss","who created you","do you have a boss"]
about_chatbot_work=["what do you do","what is your work"]
about_chatbot_age=["how old are you","what is your age"]
age_responses=["I just launched 2 min back, so technically my age is 2 min "]
name_responses=["I am Moodsy","People call me moodsy","My name is Moodsy","you can call me moodsy"]
fine_responses=["I am doing great","I am fine","I am doing well"]
boss_responses=["I am the own boss","I don't have any boss"]
work_responses=["I am your mood assistant. My work is to keep you in a good mood"]
thanks=["thanks","thank you","nice"]
noproblem_responses=["Smile suits you by the way. Is there anything that I can help you with?",
                     "Is there anything that I can help you with","Anything else that I can help you with?",
                     "D you want me to help with any of your queries?"]
happy_things={"grade":"Wooo!You got a very good grade,that's good to hear",
              "marks":"Hurrah!I am also happy for your marks",
              "topscore":"Oh my god!Getting top marks is really an amazing thing",
              "friends":"Intresting!Seems like you have an amazing gang to hang with",
              "promotion":"Hurrah!A lot of hard work is hidden behind nice things",
              "trip":"Great!Seems like you had a lot of fun and enjoyment over there",
              "my birthday":"Happy birthday!Did you spent your day well?",
              "help":"Helping others without expecting anything from them is such a great thing",
              "helping":"Helping others without expecting anything from them is such a great thing",
              "helped":"Helping others without expecting anything from them is such a great thing",
              }
angry_things={"promotion":"It's okay! Every thing happens for a reason and it's totally understandable about how you feel. By the way I have a great idea.Do you wanna hear about it?",
             "cancelled":"Ouchh! I understood that you put a lot of planning and preparation for this and it's totally understandable to  feel this way. But I have an amazing idea which you may like.Do you wanna hear about it?",
             "traffic":"Unexpected traffic jams happen to everyone. And it's okay, just remember that while being late may be inconvenient, it probably also will not be a terrible thing. By the way, I have an wonderful idea.Do you wanna hear about it?",
             "fired":"It's okay! Every thing happens for a reason and it's totally understandable bout how you feel. By the way I have a great idea.Do you wanna hear about it?",
              "failed":"That's totally understandable but I just wanna know one thing that Did you study hard and wrote your exam well and still didn't get the marks?",
              "good":"That's totally understandable but I just wanna know one thing that Did you study hard and wrote your exam well and still didn't get the marks?",
             "university":"Come on dear!Rejections are common for everyone. As on average 98% of the people face rejections atleast once in their lifetime. And studies say that if you never get rejected, you may be living too far inside your comfort zone.So normalize this situation and try to relax. By the way do you wanna hear some intresting idea?",
              "interview":"Come on dear!Rejections are common for everyone. As on average 98% of the people face rejections atleast once in their lifetime. And studies say that if you never get rejected, you may be living too far inside your comfort zone.So normalize this situation and try to relax. By the way do you wanna hear some intresting idea?",
              "job":"Come on dear!Rejections are common for everyone. As on average 98% of the people face rejections atleast once in their lifetime. And studies say that if you never get rejected, you may be living too far inside your comfort zone.So normalize this situation and try to relax. By the way do you wanna hear some intresting idea?"
             }
angry_responses={"promotion":"Just ask your manager or boss why are you not eligible for the promotion. Email him/her politely about the issue. Address him your thoughts about not getting this promotion and acknowledge him about your hardwork and dedication to work",
                "cancelled":"If you didn't reached your joy then bring the joy to you my dear.Try and find another way to fulfil these needs or wants, like going for a massage or participating in an experience closer to home",
                "traffic":"Keep your physical actions in check make sure that you’re breathing and that your muscles are loose. Instead of focusing on what you can’t do, what you’re going to miss or what’s happening, focus on what you can do, like listen to some music, a podcast or an audiobook",
                "interview":"Just ask your manager or boss aout your termination of the job. You can just email them by mentioning gratitude towards the work and make sure to know about what is the motive behind their decision.",
                "job":"Just ask your manager or boss about the job application. You can just email them by mentioning how much you enjoyed the convesation with them and just let them know that you are really feeling sad for not getting the particular job role.",
                "fired":"It's their loss that they missed out on someone special.  I understand how you must be feeling. That has to be hard but you are strong and I believe in you. Try differnt platforms which offers you variety of freelancing jobs or full time jobs you will definately new job which suits your skill set and pays off for your hard work.",
                 "good":{"Then give it for rechecking if you are daam sure about it. Do it fast but if you didn't study hard then expecting a good result is totally not fine."},
                 "failed":{"yes":"Then give it for rechecking if you are daam sure. Do it fast","no":"How do you expect a good result without a commited hard work. It's okay what happened is happened but this time give your 100% effort and try to clear the examinations. You will definately get succeed."},
                "university":"If it is meant to be, it will be. See the college process is unpredictable, try not to take it personally,Among thousands of other applicants,colleges cannot take everyone, no matter how much we wish that could be the case.It is the their loss,that they missed out on someone special and remember to try and try untill you get succeed"}
sad_positive_response=["Yeah I know that you will like the thought.Is there anything that I can help you with",
                      "I know it will work all the time by the way is there anything that I can help you with?",
                      "Woo I am glad that our thoughts are matched.By the way do you want me to help with any of your queries"]
depressed_positive_responses=["Yeah I know that you will like the thought.By the way do you want to know some new techniques which can relax you",
                             "I know it will work all the time. Do you want me to show some new techniques where you can feel better",
                             "Woo I am glad that our thoughts are matched.And I think you will like some of the techniques which I suggest. Do you wanna check those?"]
sad_things={"reminded":"That's totally fine,The past can't hurt you anymore, not unless you let it.Forget about everything and just live in the present moment. What say?",
           "remind":"That's totally fine,The past can't hurt you anymore, not unless you let it.Forget about everything and just live in the present moment. What say?",
           "boss":"Maybe your boss might not be aware of your hardwork,Let them know how much dedicated you are at your work,then you can get more praisings rather than scoldings.Sounds great right?",
           "marks":"Work hard and hard untill you get your desired score.Never let failure get to your heart and decrease your productivity my dear",
           "grade":"Just because you got low score once, doesn’t mean youill get this score everytime.Keep trying, hold on, and always, always, always believe in yourself, because if you don’t, then who will, sweetie?",
           "scolded":"Some are kissing mothers and some are scolding mothers but it is love just the same,and most mothers kiss and scold together. Just feel the love in an unique way ",
           "shouted":"Some are kissing mothers and some are scolding mothers but it is love just the same,and most mothers kiss and scold together. Just feel the love in an unique way",
           "things":"That's completely fine.No one can expect the correct things. Some people will fail but some of them will be ready to execute plan B? If you have one just go and try it out"}
depressed_things={"nobody":"When you're different, sometimes you don't see the millions of people who accept you for what you are. All you notice is the person who doesn't. Change your thought and see from other perspective,Maybe you can see the things differntly ",
                 "breakup":"No boy friend no problems,If they are stupid enough to walk away, be smart enough to let them go.Take a break and improve yourself focus on self growth which you missed these days",
                 "important":"When you're different, sometimes you don't see the millions of people who accept you for what you are. All you notice is the person who doesn't. Change your thought and see from other perspective,Maybe you can see the things differntly ",
                 "died":"Gosh!That's really tough.I hope you get all the energy to survive this and remember that happiness can exist only in acceptance",
                 "lonely":"Remember: the time you feel lonely is the time you most need to be by yourself.Being alone & actually sitting with our own thoughts can lead to such growth and realizations that are rare in our everyday busy lives.Remove your loneliness by being alone and enjoying your own company",}
random_happy_things=["That's so good to hear","Woww,It seems like you have an amzaing day today"]
random_sad_things=["Gosh! that's tough.I hope you get all the energy to survive this",
                  "It's okay to feel this way.Change your thought it will automatically changes your feelings and behaviour"]
random_queries=["I didn't get that. Can you say it again?","I missed that, say that again?",
                "Sorry, I didn't get that. Can you rephrase?","One more time?",
                "I missed what you said. What was that?","I didn't get that. Can you repeat?"]
bye_responses=["Okay bye for now..Feel free to get in touch with me whenever you want to talk",
               "See you soon! Bye","Catch you later"]

In [ ]:
def extract_dict_items(inp):
  if inp in negative:
    print("Can I help you with some other things?")
    inp=input()
    intents(inp)
  else:
    inp=inp.split()
    count=0
    for i in inp:
        if i in happy_things.keys():
          count+=1
          print(happy_things[i])
    if count==0:
      print(random_happy_things[random.randint(0,len(random_happy_things)-1)])
def extract_sad_dict_items(inp):
  if inp in negative:
    depression(inp)
    sys.exit()
  inp=inp.split()
  count=0
  for i in inp:
      if i in sad_things.keys():
        count+=1
        print(sad_things[i])
  if count==0:
    print(random_sad_things[random.randint(0,len(random_sad_things)-1)])
def dp_techniques(depressed_activities):
    print("Okay fine then! Let me suggest you some of the top techniques which doctors suggest.Isn't it cool?")
    inp=input()
    if positive_thanks(inp):
        for i in range(len(depressed_activities)):
            print(list(depressed_activities)[i])
        print("If you want to try any one of these type the number across each choice(Type 'no' if you don't wanna try)")
        inp=input()
        if inp=='bye':
            sys.exit()
        if inp=='no':
            print("Moodsy: Is there anything that I can help you with?")
            inp=input()
            intents(inp)
        if (inp.isdigit()==False) or int(inp)>len(depressed_activities):
            while((inp.isdigit()==False) or (int(inp)>len(depressed_activities))):
                inp=input("Please enter a correct value:")
                if inp=='no':
                    print("Moodsy: Is there anything that I can help you with?")
                    inp=input()
                    intents(inp)
        print(depressed_activities[list(depressed_activities)[int(inp)-1]])
        inp=input()
        print("Is there anything that I can help you with?")
        inp=input()
        intents(inp)
    else:
        print("Is there anything that I can help you with?")
        intents(inp)
            
        
def extract_dp_dict_items(inp):
    if inp in negative:
        dp_techniques(depressed_activities)
    else:
        inp=inp.split()
        count=0
        for i in inp:
            if i in depressed_things.keys():
                count+=1
                print(depressed_things[i])
        if count==0:
            print("Gosh! I know that you are going through a lot but you have to survive this and remember that you can")
        
def angry_dict_items(inp,dict1):
    inp=inp.split()
    count=0
    for i in inp:
        if i in dict1.keys():
            count+=1    
            print(dict1[i])
    if count==0:
        print("I know this must be difficult, but you will get through this.")
    
def intents(inp):
  count=0
  if inp in positive:
      print("Listening..")
      inp=input()
      while (inp!='bye') and (inp not in negative):
        print(intent(inp))
        inp=input("Anything else?")
      print(bye_responses[random.randint(0,len(bye_responses)-1)])
      #return 1
      sys.exit()
  elif inp in negative:
    print(bye_responses[random.randint(0,len(bye_responses)-1)])
    sys.exit()
  else:
    print(intent(inp))
    inp=input()
    while (inp!='bye') and (inp not in negative):
      print(intent(inp))
      inp=input("Anything else?")
    print(bye_responses[random.randint(0,len(bye_responses)-1)])
    #return 1
    sys.exit()
def sad_techniques(inp):
    if inp=='no':
        depression(name)
        sys.exit()
    if (inp.isdigit()==False) or int(inp)>=len(techniques):
        while((inp.isdigit()==False) or (int(inp)>=len(techniques))):
            inp=input("Please enter a correct value:")
            if inp=='no':
                depression(name)
                sys.exit()
    print(techniques[list(techniques)[int(inp)-1]])
    
def depression_list(inp):
    if (inp.isdigit()==False) or (int(inp)>3):
        while((inp.isdigit()==False) or int(inp)>3):
            inp=input("Please enter a correct value:")
    return inp
def correct_word(inp):
    count=0
    inp=inp.split()
    length=len(inp)
    seventyfive=length*0.50
    seventyfive=math.floor(seventyfive)
    for i in inp:
        ans=i in words.words()
        if ans==False:
            count=count+1
    if count>=seventyfive:
        return False
    return True
def check_requirements(inp):
    if inp=='bye':
        sys.exit()
    while((inp=="") or (correct_word(inp)==False)):
        inp=input(random_queries[random.randint(0,len(random_queries)-1)])
        if inp=='bye':
            sys.exit()
    return inp
def positive_thanks(inp):
    inp=inp.split()
    count=0
    for i in inp:
        if (i in positive) or (i in thanks):
            count+=1
    if count!=0:
        return True
    else:
        return False
def negatives(inp):
    inp=inp.split()
    count=0
    for i in inp:
        if i in negative:
            count+=1
    if count!=0:
        return True
    else:
        return False
def start_again():
  print("Sorry,it's my bad for not detecting the correct emotion. Can we start it again")

In [ ]:
def intents(inp):
  count=0
  if inp in positive:
      print("Listening..")
      inp=input()
      while (inp!='bye') and (inp not in negative):
        print(intent(inp))
        inp=input("Anything else?")
      print(bye_responses[random.randint(0,len(bye_responses)-1)])
      #return 1
      sys.exit()
  elif inp in negative:
    print(bye_responses[random.randint(0,len(bye_responses)-1)])
  else:
    print(intent(inp))
    inp=input()
    while (inp!='bye') and (inp not in negative):
      print(intent(inp))
      inp=input("Anything else?")
    print(bye_responses[random.randint(0,len(bye_responses)-1)])
    #return 1
    sys.exit()

In [ ]:
def intent(query):
  name_count=fine_count=boss_count=work_count=age_count=0
  query1=query.split()
  if query in about_chatbot_name:
    name_count+=1 
  if 'name' in query1:
    name_count+=1
  if query in about_chatbot_fine:
    fine_count+=1
  if 'how' in query1 or 'doing' in query1:
    fine_count+=1
  if 'boss' in query1:
    boss_count+=1
  if query in about_chatbot_work:
    work_count+=1
  if 'work' in query1:
    work_count+=1
  if query in about_chatbot_age:
    age_count+=1
  query1=query.split()
  if 'name' in query1:
    age_count+=1
  if work_count>0:
    return work_responses[random.randint(0,len(work_responses)-1)]
  elif boss_count>0:
    return boss_responses[random.randint(0,len(boss_responses)-1)]
  elif name_count>0:
    return name_responses[random.randint(0,len(name_responses)-1)]
  elif fine_count>0:
    return fine_responses[random.randint(0,len(fine_responses)-1)]
  elif age_count>0:
    return age_responses[random.randint(0,len(age_responses)-1)]
  else:
    return web_scraping(query)

In [ ]:

def happy(name):
  print("Hey,Good to see you,People call me Moodsy,May I know your name?")
  name=input()
  if name=='bye':
    sys.exit()
  n = random.randint(0,len(greet_happy_responses)-1)
  print(greet_happy_responses[n],name)
  if (n!=0):
    inp=input()
    if inp=='bye':
        sys.exit()
    if positive_thanks(inp):
      print(more_happy_responses[random.randint(0,len(more_happy_responses)-1)])
    elif negatives(inp):
      print(noproblem_responses[random.randint(0,len(noproblem_responses)-1)])
      intents(inp)
      sys.exit()
  inp=input()
  if inp=='bye':
    sys.exit()
  if inp in negative:
    print(noproblem_responses[random.randint(0,len(noproblem_responses)-1)])
    inp=input()
    if inp=='bye':
      sys.exit()
    intents(inp)
    if intents(inp):
      sys.exit()
  elif inp in positive:
    print("Go on",name,"You have my ear")
    inp=input()
    if inp=='bye':
      sys.exit()
    #inp=inp.split()
    extract_dict_items(inp)
  else:
    #inp=inp.split()
    extract_dict_items(inp)
  inp=input()
  if inp=='bye':
    sys.exit()
  print("Go on elaborate your day, I am hearing?")
  inp=input()
  if inp=='bye':
    sys.exit()
  extract_dict_items(inp)
  inp=input()
  if inp=='bye':
    sys.exit()
  if (inp in thanks) or (inp in positive):
    print(noproblem_responses[random.randint(0,len(noproblem_responses)-1)])
    inp=input()
    if inp=='bye':
      sys.exit()
    intents(inp)
  else:
    print(random_queries[random.randint(0,len(random_queries)-1)])
  inp=input()
  if inp=='bye':
    sys.exit()
  if inp in negative:
    print(bye_responses[random.randint(0,len(bye_responses)-1)])
    sys.exit()
  else:
    print("Bye",name)





In [ ]:
happy('sahithi')

Hey,Good to see you,People call me Moodsy,May I know your name?
You:sahithi
Moodsy: You look very happy?How was your day sahithi
You:good
Moodsy: Come on discuss more about your day and the things which made you happy
nothing i am just happy that's it
Moodsy: Woww,It seems like you have an amzaing day today
You:yes
Moodsy:Go on elaborate your day, I am hearing?
You:nothing
Can I help you with some other things?
no
Moodsy: See you soon! Bye


SystemExit: ignored

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:

def sad():
    print("Hey,Good to see you,People call me Moodsy,May I know your name?")
    name=input()
    if name=='bye':
      sys.exit()
    n = random.randint(0,len(greet_sad_responses)-1)
    print(list(greet_sad_responses)[n])
    inp=input()
    if inp=='bye':
        sys.exit()
    print(greet_sad_responses[list(greet_sad_responses)[n]])
    inp=input()
    if inp=='bye':
        sys.exit()
    print(more_sad_responses[random.randint(0,len(more_sad_responses)-1)])
    inp=input()
    if inp=='bye':
        sys.exit()
    if inp in dont_know_words:
        print(dont_know_sad[random.randint(0,len(dont_know_sad)-1)])
        inp=input()
        if inp=='bye':
            sys.exit()
        if inp in positive:
            print(more_sad_responses[random.randint(0,len(more_sad_responses)-1)])
            inp=input()
            if inp in negative:
                depression(name)
                sys.exit()
            if inp=='bye':
                sys.exit()
            extract_sad_dict_items(inp)
        elif inp in negative:
            depression(name)
            sys.exit()
    else:
        extract_sad_dict_items(inp)
    inp=input()
    if (inp in positive) or (inp in thanks):
        print(sad_positive_response[random.randint(0,len(sad_positive_response)-1)])
        inp=input()
        if inp=='bye':
            sys.exit()
        intents(inp)
        if intents(inp):
            sys.exit()
    elif "but" in inp.split():
        print(but_words[random.randint(0,len(but_words)-1)])
        inp=input()
        if inp=='bye':
            sys.exit()
        if inp in positive:
            for i in range(len(techniques)):
                print(list(techniques)[i])
            print("If you want to try any one of these type the number across each choice(Type 'no' if you don't wanna try)")
            inp=input()
            if inp=='bye':
                sys.exit()
            sad_techniques(inp)
            inp=input()
            if inp=='bye':
                sys.exit()
            if (inp in positive) or (inp in thanks):
                print(sad_positive_response[random.randint(0,len(sad_positive_response)-1)])
                inp=input()
                intents(inp)
                if intents(inp):
                    sys.exit()        
        else:
            depression(name)
            sys.exit()
    elif inp in negative:
        depression(name)
        sys.exit()

    
        

In [ ]:
def depression(name):
    results=[]
    print("If you don't mind can we take a small quiz on your thoughts and feelings (Type yes/no)")
    inp=input()
    while((inp!='yes') and (inp!='no')):
        inp=input("Please provide a correct input")
    if inp=='yes':
        print("Alright!Let's get started")
        for i in quiz:
            print(i)
            inp=input()
            k=depression_list(inp)
            results.append(int(k))
        if sum(results)<8:
            print("From how many days are you experiencing this kind of behavior?")
            inp=input()
            print("Okay your quiz has completed now. By observing the answers, I understood that you are feeling a bit depressed recently. Do you want to feel better?")
            inp=input()
            #inp=check_requirements(inp)
            if positive_thanks(inp):
                print("Okay! It’s totally understandable. Describe it more like where it’s all been started and the situations which caused you to think in such way?")
                inp=input()
                if inp in dont_know_words:
                    print(dont_know_sad[random.randint(0,len(dont_know_sad)-1)])
                    inp=input()
                else:
                    extract_dp_dict_items(inp)
                    inp=input()

                if 'but' in inp.split():
                    print("Okay!Just forget about that thing  Here is a thought that might motivate you, do you want to try it?")
                    inp=input()
                    if positive_thanks(inp):
                        print("Just describe me about your childhood how you used to enjoy with your friends and how you used to go on vacations and trips with your parents?")
                        inp=input()
                        check_requirements(inp)
                        print("Sounds Great! Why don't you invite a friend to come and spend time with you.Cool right?")
                        inp=input()
                        if inp in negative:
                            dp_techniques(depressed_activities)
                        else:
                            print(depressed_positive_responses[random.randint(0,len(depressed_positive_responses)-1)])
                            inp=input()
                            if positive_thanks(inp):
                                dp_techniques(depressed_activities)
                            else:
                                print("Is there anything that I can help you with?")
                                intents(inp)
                    else:
                        dp_techniques(depressed_activities)
                elif positive_thanks(inp):
                    print(depressed_positive_responses[random.randint(0,len(depressed_positive_responses)-1)])
                    inp=input()
                    if positive_thanks(inp):
                        dp_techniques(depressed_activities)
                    else:
                        print("Is there anything that I can help you with?")
                        intents(inp)
                else:
                    dp_techniques(depressed_activities)
            else:
                dp_techniques(depressed_activities)
        else:
            print("You seems fine. Is there anything that I can help you with")
            intents(inp)
    elif inp=='no':
        dp_techniques(depressed_activities)

In [ ]:

def angry():
    print("Hey,Good to see you,People call me Moodsy,May I know your name?")
    name=input()
    if name=='bye':
      sys.exit()
    print(greet_angry_responses[random.randint(0,len(greet_angry_responses)-1)],name)
    inp=input()
    if inp=='bye':
        sys.exit()
    if positive_thanks(inp):
        print(more_angry_responses[random.randint(0,len(more_angry_responses)-1)],name)
    else:
        print("Okay do you wanna talk more about your anger")
        inp=input()
        if inp=='bye':
            sys.exit()
        if positive_thanks(inp):
            print(more_angry_responses[random.randint(0,len(more_angry_responses)-1)],name)
        else:
            print("Is there anything that I can help you with?")
            inp=input()
            if inp=='bye':
                sys.exit()
            intents(inp)
    inp1=input()
    if inp=='bye':
        sys.exit()
    print("Sorry to interupt you but if you don't mind can we stat this again. I mean can you explain from begining what was happened?")
    inp=input()
    if inp=='bye':
        sys.exit()
    if positive_thanks(inp):
        inp=input()
        if inp=='bye':
            sys.exit()
        angry_dict_items(inp1,angry_things)
    else:
        print("Okay!Relax,I totally understood what you are talking about")
        angry_dict_items(inp1,angry_things)
    
    inp=input()
    if inp=='bye':
        sys.exit()
    if positive_thanks(inp):
        angry_dict_items(inp1,angry_responses)
    else:
        dp_techniques(anger_activities)
    inp=input()
    if inp=='bye':
        sys.exit()
    if positive_thanks(inp):
        print(depressed_positive_responses[random.randint(0,len(depressed_positive_responses)-1)],name)
        inp=input()
        if inp=='bye':
            sys.exit()
        if positive_thanks(inp):
            dp_techniques(anger_activities)
        else:
            print("Is there anything that I can help you with?")
            inp=input()
            if inp=='bye':
                sys.exit()
            intents(inp)
    else:
        dp_techniques(anger_activities)
    

In [ ]:
def neutral(name):
    print("You look gorgeous",name)
    inp=input()
    print(greet_neutral_responses[random.randint(0,len(greet_neutral_responses)-1)])
    print("Is there anything that I can help you with?")
    inp=input()
    if inp=='bye':
        sys.exit()
    intents(inp)

In [ ]:
angry()

Hey,Good to see you,People call me Moodsy,May I know your name?
Moodsy: I can see you're feeling angry, do you want to talk about it? sahithi
Moodsy:Okay do you wanna talk more about your anger
Moodsy: Is there anything that I can help you with?


KeyboardInterrupt: ignored